<a href="https://colab.research.google.com/github/venkatbalineni/NLP/blob/NLP_develop/DL_NLP_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html


Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.5MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62e86000 @  0x7f75aecfc2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [5]:
!pip install fastai

    100% |████████████████████████████████| 143kB 10.8MB/s 
    100% |████████████████████████████████| 61kB 23.5MB/s 
    100% |████████████████████████████████| 163kB 14.4MB/s 
    100% |████████████████████████████████| 112kB 27.5MB/s 
    100% |████████████████████████████████| 1.9MB 14.6MB/s 
    100% |████████████████████████████████| 23.3MB 1.7MB/s 
  Running setup.py bdist_wheel for bottleneck ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/f2/bf/ec/e0f39aa27001525ad455139ee57ec7d0776fe074dfd78c97e4
Successfully built bottleneck
spacy 2.0.16 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
torchvision-nightly 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: thinc 6.12.1
    Uninstalling thinc-6.12.1:
      Successfully uninstalled thinc-6.12.1
  Found existing installation: spacy 2.0.18
    Uninstalling spacy-2.0.18:
      Successfully

In [7]:
!pip install dataclasses

In [0]:
#Import libraries
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [31]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle = True, random_state = 1, remove = ('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
df = pd.DataFrame({'label': dataset.target, 'text': dataset.data})

In [0]:
df.head

In [0]:
#df[df['label'] == 10]
#converting this into a binary classification problem by only keeping two classes 1 & 10
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)

In [29]:
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [0]:
#Retaining only alphabets and removing othere
df['text'] = df['text'].str.replace("[^a-zA-Z]"," ")

In [0]:
#Tokenization
tokenized_doc = df['text'].apply(lambda x: x.split())

#Remove Stopwords
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item.lower() not in stop_words])

#Detokenized doc
detokenized_doc = []

for i in range(len(df)):
  t = ' '.join(tokenized_doc[i])
  detokenized_doc.append(t)
 
df['text']=detokenized_doc

In [0]:
#df.head
#Creating training and validation data sets
df_train, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [41]:
df_train.shape, df_val.shape

((710, 2), (474, 2))

In [0]:
#prepare data for language model
data_lm = TextLMDataBunch.from_df(train_df = df_train, valid_df = df_val, path = '')

#classification model data
data_class = TextClasDataBunch.from_df(train_df = df_train, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs = 32, path = '')

In [0]:
learn = language_model_learner(data_lm, pretrained_model= URLs.WT103, drop_mult=0.7)

In [46]:
#learning object with learning rate 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.537339,6.249417,0.160714


In [0]:
learn.save_encoder('ft_enc')

In [0]:
#creating classifier with finetuned encoding
learn = text_classifier_learner(data_class, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [49]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.533456,0.313429,0.864979


In [50]:
#get Predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis=1)
pd.crosstab(predictions, targets)


col_0,0,1
row_0,,
0,219,49
1,15,191
